In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

### Change Webpage Link Of Audiable As Per Category For Scrap 

In [3]:
# change web link as per your category
web  = 'https://www.audible.in/adblbestsellers?ref=a_search_t1_navTop_pl1cg0c1r0&pf_rd_p=88b2dc24-01ab-437a-bf8e-1faf3da486fb&pf_rd_r=VV04PSD0ZPFDF5KA7BK3&pageLoadId=jmu5LLlZOPWjNC6o&creativeId=2e6787a2-0cd0-4a6e-afe0-05766cd505e5'
path = '/home/priteshs/Downloads/chromedriver'

In [73]:
options = Options()
options.headless = True
options.add_argument('window-size=1920x1080')


driver = webdriver.Chrome(path)
driver.get(web)
driver.maximize_window()

# pagination
pagination = driver.find_element_by_xpath('//ul[contains(@class,"pagingElements")]')
pages = pagination.find_elements_by_tag_name('li')
last_page = int(pages[-2].text)

current_page = 1

book_title = []
book_author = []
book_length = []
book_rating = []

while current_page <= last_page:

    # item per page is 50
    pages = Select(driver.find_element_by_xpath('//select[@name="pageSize"]'))
    pages.select_by_visible_text(driver.find_elements_by_xpath('//select[@name="pageSize"]/option')[-1].text)

    # sleep after change item per page to reload contain of site
    # implicit wait   
    # time.sleep(2)
    # container = driver.find_element_by_class_name('adbl-impression-container ')
    # products = container.find_elements_by_xpath('.//li[contains(@class,"productListItem")]')

    # explicit wait
    container = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME,'adbl-impression-container ')))
    products = WebDriverWait(container,3).until(EC.presence_of_all_elements_located((By.XPATH,'.//li[contains(@class,"productListItem")]')))
    
    for book in products:
        book_title.append(book.find_element_by_xpath('.//h3[contains(@class,"bc-heading")]/a').text)
        book_author.append(book.find_element_by_xpath('.//li[contains(@class,"authorLabel")]').text.split(':')[1])
        book_length.append(book.find_element_by_xpath('.//li[contains(@class,"runtimeLabel")]').text.split(':')[1])
        book_rating.append(book.find_element_by_xpath('.//li[contains(@class,"ratingsLabel")]/span[contains(@class,"bc-pub-offscreen")]').text)

    current_page += 1
    try:
        next_page = driver.find_element_by_xpath('//span[contains(@class,"nextButton")]')
        next_page.click()
    except:
        pass


# print(f'book_title: {len(book_title)}, book_author: {len(book_author)}, book_length: {len(book_length)}, book_rating: {len(book_rating)}')

df = pd.DataFrame({'title':book_title, 'author':book_author, 'length':book_length, 'rating':book_rating})

df.to_csv('book_pagination.csv',index=False)